# Omdena  - Milan Chapter Agrifoods
## AI for Sustainable agri-food systems: use of Satellite Imagery
### Exploratory analysis of cereals production in Italy 2006-2022
#### Author: Maria Fisher 


The main objective of this study is to have gather information about crop production in Italy for the period of 2006-2022. 

Crop dataset used in this study was downloaded from the Italian National Institute of Statistics (Istat).



In [ ]:
import warnings 
warnings.filterwarnings("ignore")

import os
import pandas as pd
pd.options.display.float_format = "{:.2f}".format
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import seaborn as sns 
import scipy 
import sklearn
import geopandas as gpd
import pgeocode
import folium
import sys
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot, plot

In [ ]:
cereals = pd.read_csv('./Italy_crop_data/cereals_legumes.csv',skipinitialspace=True)
cereals.head()

## Pre-processing dataset 

In [ ]:
# Drop Columns
cereals = cereals.drop(columns =['ITTER107','TIPO_DATO5','AGRI_MADRE', 'TIME', 'Flag Codes','Flags' ])
cereals

In [ ]:
# Rename Columns
cereals = cereals.rename(columns = {'Select time':'Year', 'Type of crop':'Type_crop', 'Data type':'Data_type', 'Territory':'City'})


In [ ]:
def show_info(cereals):
    print('DATASET SHAPE: ', cereals.shape, '\n')
    print('-'*50)
    print('FEATURE DATA TYPES:')
    print(cereals.info())
    print('\n', '-'*50)
    print('NUMBER OF UNIQUE VALUES PER FEATURE:', '\n')
    print(cereals.nunique())
    print('\n', '-'*50)
    print('NULL VALUES PER FEATURE')
    print(cereals.isnull().sum())
show_info(cereals)

## Cities in Italy producing cereals

In [ ]:
cereals.City.unique()


In [ ]:
cereals.Data_type.unique()

## Select only Values for total production - quintals
In general crop production is reported in tonnes per hectare , however Istat (http://dati.istat.it) does not give variables definition, so we have assumed that the column 'Value' for rows containing data for 'harvested production - quintals', are values for total production of cereals  = yield(in quintal/ha). 

In [ ]:
cereals_total_prod = cereals[cereals['Data_type'] == 'total production - quintals ']
cereals_total_prod.head()

In [ ]:
cereals_total_prod['Value'] = cereals_total_prod['Value']/10
cereals_total_prod

In [ ]:
cereals_total_prod = cereals_total_prod.rename(columns = {'Value':'Production_tonnes'})
cereals_total_prod

In [ ]:
cereals_total_prod =cereals_total_prod.drop(columns =['Data_type' ])
cereals_total_prod

## Total cereal production in 2006-2022

In [ ]:
plt.figure(figsize= (10,5))
sns.barplot(x= 'Year', y= 'Production_tonnes',data = cereals_total_prod,palette='coolwarm')
plt.title('Total cereal and legume production 2006-2022')
plt.xlabel('Year')
plt.ylabel('Total production (tonnes)')
plt.show()



## Cereal production by Cities 

In [ ]:
cereals_total_prod_region = cereals_total_prod.groupby(by = cereals_total_prod.City)['Production_tonnes','City'].sum().reset_index().sort_values(by = 'Production_tonnes', ascending = False).head(10)
cereals_total_prod_region

In [ ]:
plt.figure(figsize= (10,5))
sns.barplot(x=cereals_total_prod_region['Production_tonnes'],y= cereals_total_prod_region['City'], orient='h', palette='coolwarm');
plt.title('Total cereal and legume production 2006-2022 by City')
plt.xlabel('Total production')
plt.ylabel('Cities')
plt.show()

## Italy geographic areas is organized in regions, provinces or comunes. 
### The regions are classified as follow:


#### Sud/Mezzogiorno:  

Abruzzo, Apulia/Puglia, Basilicata, Calabria, Campania, Molise, Sicily 

* Abruzzo: L'Aquila, Pescara, Chieti, Teramo

* Apulia/Puglia: Foggia, Bari, Taranto, Brindisi, Lecce, Barletta-Andria-Trani

* Basilicata: Matera, Potenza

* Calabria: Crotone, Vibo, Valentia, Cosenza, Catanzaro, Reggio di Calabria

* Campania: Benevento, Caserta, Napoli, Avellino, Salerno

* Molise: Campobasso, Isernia

* Sicilia: Messina,Siracusa, Agrigento, Caltanissetta, Trapani, Enna, Palermo, Catania, Ragusa

* Sardegna: Oristano, Carbonia-Iglesias, Olbia-Tempio, Ogliastr, Medio Campidano, Sassari, Nuoro, Cagliari



#### Centro: 

Toscana, Umbria, Marche, Lazio

* Toscana: Pistoia, Firenze, Massa-Carrara, Lucca, Arezzo, Livorno, Pisa, Grosseto, Siena, Prato

* Umbria: Perugia, Terni   

* Marche: Ancona, Macerata, Ascoli Piceno, Pesaro e Urbino, Fermo

* Lazio:  Roma, Viterbo, Rieti, Latina, Frosinone


#### Nord-ovest: 

Piemonte, Valle d’Aosta, Lombardia, Liguria

* Liguria: Imperia, Savona, Genova, La Spezia

* Lombardia: Como, Varese, Milano, Pavia, Bergamo, Brescia, Sondrio, Cremona, Mantova, Monza e Della Brianza, Lecco,Lodi

* Piemonte: Vercelli, Novara, Torino, Cuneo, Asti, Alessandria, Biella, Verbano-Cusio-Ossola

* Valdaosta:  Valle d'Aosta


#### Nord-est:

Trentino-Alto Adige, Veneto, Fiuli-Venezia Giulia, Emilia-Romagna


* Trentino: Bolzano/Bozen, Trento

* Veneto:  Belluno, Verona, Vicenza, Rovigo, Treviso, Venezia, Padova

* Friuli: Udine, Gorizia, Trieste, Pordenone

* Emilia-Romagna: Parma, Reggio Nell'Emilia, Piacenza, Forli'-Cesena, Modena, Bologna, Ferrara, Ravenna, Rimini











## Cereals highest production 2006-2022

In [ ]:
print(cereals_total_prod.Type_crop.max())
print(cereals_total_prod.Type_crop.value_counts())
print(cereals_total_prod.Type_crop.nunique())



Dataset shows there are 29 different types of cereals cultivated in Italy. Ten crop produced are Common wheat, Durum wheat, Potatoes, Barley, Maize, Beans, Chick-peas, Rye, Rice and Oats. 

In [ ]:
cereals_total_prod.describe().astype(int)

In [ ]:
# Rename name of crops 
cereals_total_prod = cereals_total_prod.replace('oats and spring cereal mixtures (mixed grain other than maslin)','oats mix')
cereals_total_prod = cereals_total_prod.replace('rye and winter cereal mixtures (maslin)','rye mix')
cereals_total_prod = cereals_total_prod.replace('spring cereal mixtures (mixed grain other than maslin)','cereal mix')
cereals_total_prod = cereals_total_prod.replace('common spring wheat and spelt','c-spr-wheat&spelt')
cereals_total_prod = cereals_total_prod.replace('common winter wheat and spelt','c-wint-wheat&spelt')
cereals_total_prod = cereals_total_prod.replace('winter cereal mixtures (maslin)','wint-cereal-mix')
cereals_total_prod = cereals_total_prod.replace('dried kidney bean','dry-k-bean')
cereals_total_prod = cereals_total_prod.replace('common wheat','c-wheat')
cereals_total_prod = cereals_total_prod.replace('durum wheat','d-wheat')
cereals_total_prod = cereals_total_prod.replace('broad bean','bro-bean')
cereals_total_prod = cereals_total_prod.replace('grain maize','maize')



 

In [ ]:
plt.figure(figsize=(10,5))
cereals_total_prod ['Type_crop'].value_counts().plot.bar()
plt.title('Total cereal and legume production 2006-2022 by crop')
plt.ylabel('Total production (tonnes)')
plt.show()


## Subseting data

In [ ]:
cereals_top10 = cereals_total_prod.apply(lambda row: row[cereals_total_prod['Type_crop'].isin(['barley','oats', 
                                         'd-wheat','c-wheat', 'maize', 'potatoes','dry-k-bean',
                                          'bro-bean','chick-peas','rye'])])

cereals_top10.head()

In [ ]:
plt.figure(figsize=(8,6))

sns.boxplot(
    data=cereals_top10, y="Type_crop", x="Production_tonnes",
    notch=True, showcaps=False,
    flierprops={"marker": "."},
    boxprops={"facecolor": (.9, .6, .8, .5)},
    medianprops={"color": "black"},
)

plt.title('Ten cereals and legumes produced Italy')
plt.ylabel('Type of Crop')
plt.xlabel('Total harvested')
plt.show()

## Vizualization of ten type of cereals and legumes produced in Italy in the period of 2006-2022 

In [ ]:
fig, axs = plt.subplots(3,2, figsize=(13, 10))
plt.ylabel(" ")

plot1 = cereals_top10[cereals_top10.Year == 2006].groupby('Type_crop').size().nlargest(10).plot(kind='barh', color='#6b6ecf', title='2006', ax=axs[0,0])
plot2 = cereals_top10[cereals_top10.Year == 2007].groupby('Type_crop').size().nlargest(10).plot(kind='barh', color='#6b6ecf', title='2007', ax=axs[0,1])
plot3 = cereals_top10[cereals_top10.Year == 2008].groupby('Type_crop').size().nlargest(10).plot(kind='barh', color='#6b6ecf', title='2008', ax=axs[1,0])
plot4 = cereals_top10[cereals_top10.Year == 2009].groupby('Type_crop').size().nlargest(10).plot(kind='barh', color='#6b6ecf', title='2009', ax=axs[1,1])
plot5 = cereals_top10[cereals_top10.Year == 2010].groupby('Type_crop').size().nlargest(10).plot(kind='barh', color='#6b6ecf', title='2010', ax=axs[2,0])
plot6 = cereals_top10[cereals_top10.Year == 2011].groupby('Type_crop').size().nlargest(10).plot(kind='barh', color='#6b6ecf', title='2011', ax=axs[2,1])

for ax in axs.flat:
    ax.bar_label(ax.containers[0], fmt='%.0f', label_type='edge', padding=2)
    ax.margins(x=0.1)
    ax.set_ylabel(' ')
    sns.despine()

plt.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(3,2, figsize=(13, 10))


plot7 = cereals_top10[cereals_top10.Year == 2012].groupby('Type_crop').size().nlargest(10).plot(kind='barh', color='#6b6ecf', title='2012', ax=axs[0,0])
plot8 = cereals_top10[cereals_top10.Year == 2013].groupby('Type_crop').size().nlargest(10).plot(kind='barh', color='#6b6ecf', title='2013', ax=axs[0,1])
plot9 = cereals_top10[cereals_top10.Year == 2014].groupby('Type_crop').size().nlargest(10).plot(kind='barh', color='#6b6ecf', title='2014', ax=axs[1,0])
plot10 = cereals_top10[cereals_top10.Year == 2015].groupby('Type_crop').size().nlargest(10).plot(kind='barh', color='#6b6ecf', title='2015', ax=axs[1,1])
plot11 = cereals_top10[cereals_top10.Year == 2016].groupby('Type_crop').size().nlargest(10).plot(kind='barh', color='#6b6ecf', title='2016', ax=axs[2,0])
plot12 = cereals_top10[cereals_top10.Year == 2017].groupby('Type_crop').size().nlargest(10).plot(kind='barh', color='#6b6ecf', title='2017', ax=axs[2,1])

for ax in axs.flat:
    ax.bar_label(ax.containers[0], fmt='%.0f', label_type='edge', padding=2)
    ax.margins(x=0.1)
    ax.set_ylabel(' ')
    sns.despine()
    
plt.tight_layout()
plt.show()

In [ ]:

fig, axs = plt.subplots(2,3, figsize=(25,12))

axs[1,0].set_position([0.24,0.125,0.228,0.343])
axs[1,1].set_position([0.55,0.125,0.228,0.343])
axs[1,2].set_visible(False)

plot13 = cereals_top10[cereals_top10.Year == 2018].groupby('Type_crop').size().nlargest(10).plot(kind='barh', color='#6b6ecf', title='2018', ax=axs[0,0])
plot14 = cereals_top10[cereals_top10.Year == 2019].groupby('Type_crop').size().nlargest(10).plot(kind='barh', color='#6b6ecf', title='2019', ax=axs[0,1])
plot15 = cereals_top10[cereals_top10.Year == 2020].groupby('Type_crop').size().nlargest(10).plot(kind='barh', color='#6b6ecf', title='2020', ax=axs[0,2])
plot16 = cereals_top10[cereals_top10.Year == 2021].groupby('Type_crop').size().nlargest(10).plot(kind='barh', color='#6b6ecf', title='2021', ax=axs[1,0])
plot17 = cereals_top10[cereals_top10.Year == 2022].groupby('Type_crop').size().nlargest(10).plot(kind='barh', color='#6b6ecf', title='2022', ax=axs[1,1])

for ax in axs.flat:
    ax.bar_label(ax.containers[0], fmt='%.0f', label_type='edge', padding=2)
    ax.margins(x=0.1)
    ax.set_ylabel(' ')
    sns.despine()

plt.tight_layout()

plt.show()



In [ ]:
#Crop harvested by year
plotcrop10 = sns.relplot(kind='line', data=cereals_top10, x='Year', y='Production_tonnes', hue='Type_crop', style='Type_crop', aspect=1.75)


In [ ]:
# Create columns for each type of fertilizer 
cereals_top10_pivot = cereals_top10.pivot_table(index=['City', 'Year'],aggfunc='sum', columns='Type_crop',values=['Production_tonnes'])
cereals_top10_pivot




In [ ]:

# Drop level
cereals_top10_pivot.columns = cereals_top10_pivot.columns.droplevel()
cereals_top10_pivot



In [ ]:
cereals_top10_pivot = cereals_top10_pivot.fillna(0)
cereals_top10_pivot



In [ ]:
#Rename index
cereals_top10_pivot = cereals_top10_pivot.rename_axis(None,axis=1)
cereals_top10_pivot



In [ ]:
# Reset index
cereals_top10_pivot = cereals_top10_pivot.reset_index()
cereals_top10_pivot

## Statistical analysis

In [ ]:
cereals_top10_pivot.describe()

#### Data metrics : skew, kurtosis and mode

In [ ]:
cereals_top10_pivot_stats = cereals_top10_pivot[['barley','bro-bean','c-wheat','chick-peas','d-wheat','dry-k-bean','maize','oats','potatoes','rye']]
print(cereals_top10_pivot_stats.astype(float).skew())
print(cereals_top10_pivot_stats.astype(float).kurtosis())
print(cereals_top10_pivot_stats.astype(float).mode())


### Graphs
#### Histogram and Boxplot showing data distribuion and outliers¶


### Normalizing the dataset

The data need to be normalized for further use in a modelling. Here the data will be normalize by deleting outliers using the interquartile range (IQR).


#### Calculate and print the interquartile range

In [ ]:

Q1 = cereals_top10_pivot_stats.quantile(0.25)
Q3 = cereals_top10_pivot_stats.quantile(0.75)
IQR = Q3 - Q1
print(IQR)


#### Remove outliers and plot graphs

In [ ]:
# Remove outliers
cereals_top10_pivot_stats[~((cereals_top10_pivot < (Q1 - 1.5 * IQR)) | (cereals_top10_pivot_stats > (Q3 + 1.5 * IQR))).any(axis=1)]
print(cereals_top10_pivot.shape)

In [ ]:
fig, ax = plt.subplots(5, 2, figsize = (10, 10))
sns.boxplot(x= cereals_top10_pivot_stats["barley"], color = 'white', ax = ax[0,0])
sns.distplot(cereals_top10_pivot_stats['barley'], color = 'darkblue', ax = ax[0,1])
sns.boxplot(x= cereals_top10_pivot_stats["bro-bean"],color = 'white', ax = ax[1,0])
sns.distplot(cereals_top10_pivot_stats['bro-bean'], color = 'darkblue', ax = ax[1,1])
sns.boxplot(x= cereals_top10_pivot_stats["c-wheat"],color = 'white', ax = ax[2,0])
sns.distplot(cereals_top10_pivot_stats['c-wheat'], color = 'darkblue',ax = ax[2,1])
sns.boxplot(x= cereals_top10_pivot_stats["chick-peas"], color = 'white', ax = ax[3,0])
sns.distplot(cereals_top10_pivot_stats['chick-peas'], color = 'darkblue', ax = ax[3,1])
sns.boxplot(x= cereals_top10_pivot_stats["d-wheat"], color = 'white',ax = ax[4,0])
sns.distplot(cereals_top10_pivot_stats['d-wheat'], color = 'darkblue',ax = ax[4,1])

plt.tight_layout()

In [ ]:
fig, ax = plt.subplots(5, 2, figsize = (10, 10))
sns.boxplot(x= cereals_top10_pivot_stats["maize"], color = 'white', ax = ax[0,0])
sns.distplot(cereals_top10_pivot_stats['maize'], color = 'darkblue', ax = ax[0,1])
sns.boxplot(x= cereals_top10_pivot_stats["oats"],color = 'white', ax = ax[1,0])
sns.distplot(cereals_top10_pivot_stats['oats'], color = 'darkblue', ax = ax[1,1])
sns.boxplot(x= cereals_top10_pivot_stats["potatoes"],color = 'white', ax = ax[2,0])
sns.distplot(cereals_top10_pivot_stats['potatoes'], color = 'darkblue',ax = ax[2,1])
sns.boxplot(x= cereals_top10_pivot_stats["dry-k-bean"], color = 'white',ax = ax[3,0])
sns.distplot(cereals_top10_pivot_stats['dry-k-bean'], color = 'darkblue',ax = ax[3,1])
sns.boxplot(x= cereals_top10_pivot_stats["rye"], color = 'white',ax = ax[4,0])
sns.distplot(cereals_top10_pivot_stats['rye'], color = 'darkblue',ax = ax[4,1])

plt.tight_layout()

In [ ]:
cereals_top10_pivot

### Pre-processing ferlizers dataset
fertilizers distributed - tonnes

In [ ]:
    fertilizer = pd.read_csv('fertilizer_by_prov.csv',skipinitialspace=True)
fertilizer.head()

In [ ]:
fertilizer = fertilizer.drop(columns =['ITTER107','TIPO_DATO5', 'TIME','FERTILIZZANTI','Flag Codes','Flags'])
fertilizer

In [ ]:
# change name of columns
fertilizer = fertilizer.rename(columns = {'Select time':'Year', 'Type of fertilizer':'Type_fertilizer', 'Data type':'Data_type', 'Territory':'City', 'Value':'Fertilizers_tonnes'})
fertilizer

In [ ]:
fertilizer.City.unique()

In [ ]:
fertilizer.Data_type.unique()

In [ ]:
#convert quintals to tonnes
fertilizer.loc[fertilizer['Data_type'] == 'fertilizers distributed - quintals','Fertilizers_tonnes' ] = fertilizer['Fertilizers_tonnes'] / 10



In [ ]:
fertilizer.Data_type.unique()

In [ ]:
fertilizer

### Selecting fertilizers for future analysis 

In [ ]:
print(fertilizer.Type_fertilizer.max())
print(fertilizer.Type_fertilizer.value_counts())
print(fertilizer.Type_fertilizer.nunique())

In [ ]:
# Rename name of fertilizers 
fertilizer = fertilizer.replace('organic fertilizers - straight nitrogen','organic-nitrogen')
fertilizer = fertilizer.replace('organic-mineral fertilizers - straight nitrogen','organic-nitrogen')
fertilizer = fertilizer.replace('organic-mineral fertilizers - compound','organic-mineral')
fertilizer = fertilizer.replace('organic fertilizers - compound','organic')
fertilizer = fertilizer.replace('mixed soil amendment','mix-amend')
fertilizer = fertilizer.replace('peaty soil amendment','peaty-amend')
fertilizer = fertilizer.replace('peaty amendment','peaty-amend')
fertilizer = fertilizer.replace('peat amendment','peaty-amend')
fertilizer = fertilizer.replace('two components - nitrogen-phosphorous','nitrogen-phosphorous')
fertilizer = fertilizer.replace('two components - nitrogen-potassium','nitrogen-potassium')
fertilizer = fertilizer.replace('two components - nitrogen-phosphorous','nitrogen-phosphorous')
fertilizer = fertilizer.replace('two components - phosphorus-potassium','phosphorus-potassium')
fertilizer = fertilizer.replace('vegetable soil amendment', 'organic')

In [ ]:
print(fertilizer.Type_fertilizer.max())
print(fertilizer.Type_fertilizer.value_counts())


In [1]:
fertilizer = fertilizer.apply(lambda row: row[fertilizer['Type_fertilizer'].isin(['calcium cyanamide','nitrates','organic', 'urea','nitrogen-potassium',
                                                                                  'phosphorus-potassium','ammonium sulphate','calcium cyanamide', 
                                                                                  'nitrogen-phosphorous','peaty-amend','organic-nitrogen' ])])

fertilizer.head()

NameError: name 'fertilizer' is not defined

In [ ]:

plt.figure(figsize=(10,10))
fertilizer['Type_fertilizer'].value_counts().plot.bar()
plt.title('Total fertilizers distributed 2006-2021')
plt.ylabel('Total fertilizers (tonnes)')
plt.show()


In [ ]:
# fertilizer distributed tonnes and quintals
plt.figure(figsize= (10,5))
sns.barplot(x= 'Year', y= 'Fertilizers_tonnes',data = fertilizer, palette='coolwarm')
plt.title('Fertilizers distributed (tonnes) 2006-2021')
plt.xlabel('Year')
plt.ylabel('Fertilizers total (tonnes)')
plt.show()


In [ ]:
fertilizer = fertilizer.nlargest(30, 'Fertilizers_tonnes')

In [ ]:
fertilizer= fertilizer.sort_values('Fertilizers_tonnes',ascending=False)

plt.figure(figsize= (10,5))
sns.barplot(x=fertilizer['Fertilizers_tonnes'] ,y= fertilizer ['City'], palette='coolwarm');
plt.title('Total Fertilizers distributed (tonnes) 2006-2021 by City')
plt.xlabel('Total')
plt.ylabel('Cities')
plt.show()

In [ ]:
#Fertilizers distributed by year

fertilizer_plot = sns.relplot(kind='line', data=fertilizer, x='Year', y='Fertilizers_tonnes', hue='Type_fertilizer', style='Type_fertilizer', aspect=1.75)


In [ ]:
fertilizer.head()

#### Create new dataframe with the selected type of fertilizers as columns  

In [ ]:
# Create columns for each type of fertilizer 
fertilizer_pivot = fertilizer.pivot_table(index=['City', 'Year'],aggfunc='sum', columns='Type_fertilizer',values=['Fertilizers_tonnes'])
fertilizer_pivot

In [ ]:
# Drop level
fertilizer_pivot.columns = fertilizer_pivot.columns.droplevel()
fertilizer_pivot

In [ ]:
#Rename index
fertilizer_pivot = fertilizer_pivot.rename_axis(None,axis=1)
fertilizer_pivot

In [ ]:
# Reset index
fertilizer_pivot = fertilizer_pivot.reset_index()
fertilizer_pivot

#### Join crop and fertilizers datasets 

In [ ]:
# Join both datasets 
it_crop_fertilizer = pd.merge(cereals_top10_pivot, fertilizer_pivot, on=['Year', 'City'], how='left').fillna(0)
it_crop_fertilizer

### Correlation analysis


In [ ]:

crop = it_crop_fertilizer.City.astype('category')
targets = dict(enumerate(crop.cat.categories))
it_crop_fertilizer['target']=crop.cat.codes

cor_selected = it_crop_fertilizer[['barley','oats','d-wheat','c-wheat', 'maize',
                                 'potatoes','dry-k-bean','bro-bean','chick-peas',
                                 'rye','calcium cyanamide','nitrogen-potassium','organic',
                                 'phosphorus-potassium','urea']]


In [ ]:
plt.figure(figsize=(13,10))
plt.title('Correlation cereals & legumes 2006-2021', size=10)
sns.heatmap(cor_selected.corr(), cmap='crest', center=0, annot=True)

### Correlation analysis of cereal and legumes production in Italy 2006 -2021 : 

* Maize has hight correlation with common Wheat and correlation with urea.

* Barley has hight correlation with Wheat.

* Oats has hight correlation with Durum Wheat. 

* Common Wheat has hight correlation with urea. 

* Potatoes has correlation with Wheat. 

* There is also hight correlation between urea, nitrogen-potassium, calcium and phosphorus-potassium fertilizers.


### Calculating Variance Inflation Factor (VIF) for all given features



In [ ]:
# Function to compute the VIF
def compute_vif(selected_features):
    
    y = cor_selected [selected_features]
    # the calculation of variance inflation requires a constant
    y['intercept'] = 1
    
    # create dataframe to store vif values
    vif = pd.DataFrame()
    vif["Feature"] = y.columns
    vif["VIF"] = [variance_inflation_factor(y.values, i) for i in range(y.shape[1])]
    vif = vif[vif['Feature']!='intercept']
    return vif

In [ ]:
# Features selection
selected_features = ['barley','oats','d-wheat','c-wheat', 'maize',
                                 'potatoes','dry-k-bean','bro-bean','chick-peas',
                                 'rye','calcium cyanamide','nitrogen-potassium','organic',
                                 'phosphorus-potassium','urea']

# compute vif 
compute_vif(selected_features).sort_values('VIF', ascending=False)


We have performed variance inflation factor (VIF) to detect if there is multicollinearity. Some references indicate a serious collinearity problem if VIF greater or equal to 5. 

In this analysis none of the features showed VIF higher than 5.  
 

Modelling*

# References


http://dati.istat.it

https://maps.princeton.edu/catalog/stanford-mn871sp9778

https://www.crea.gov.it/documents/68457/0/ITACONTA+2020_ENG+DEF+xweb+%281%29.pdf/95c6b30a-1e18-8e94-d4ac-ce884aef76e8?t=1619527317576

https://seaborn.pydata.org/generated/seaborn.relplot.html

https://www.statisticshowto.com/variance-inflation-factor/

https://statisticsbyjim.com/regression/multicollinearity-in-regression-analysis/